 # Regex and Prodigy
 
#Coor
 

<font size= "3">
Annotation of entities for sentences based on prodigy's format for different labels for further usage in Machine learning. Prodigy is an annotation tool based on spaCy. Please see here:
    </p>
  <a href="https://prodi.gy/ "> Prodigy </a> 
<font 3>

## Imports

In [62]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import spacy 
import re
from prodigy.util import write_jsonl



In [63]:
pwd

'C:\\Projects_28_06\\notebooks\\yeghaneh\\BZML\\notebooks'

## Read the Data

In [64]:
importVersion = '013'

In [65]:
path= '../data/01_df_v{0}.pickle'.format(importVersion)# Put the path of the data in your local machine here, consider the letter "r" before the path
dfAstroNova = pd.read_pickle(path)

# Sort based on the Chapter 

In [66]:
# Sort the data based on the chapters of the book 
dfAstroNova['chapter'] = dfAstroNova.chapter.replace("appendix b",np.nan).astype(float)  
dfAstroNova = dfAstroNova.rename_axis('MyIdx').sort_values(by = ['chapter', 'MyIdx'], ascending = [True, True])
dfAstroNova.chapter.fillna('appendix b', inplace=True)

In [67]:
dfAstroNova.reset_index(inplace=True)
dfAstroNova=dfAstroNova.drop("MyIdx",axis=1,inplace=False)
dfAstroNova=dfAstroNova.drop("html",axis=1)

In [68]:
type(dfAstroNova)

pandas.core.frame.DataFrame

In [69]:
dfAstroNova.head(5)

,text,links,italic,chapter,graphic,table,marginal,sentences,tagged
0,Chapter 1,[],[],1,[],[],[],[Chapter 1],"[[(Chapter, None), (1, NUM)]]"
1,On the distinction between the first motion an...,[],[],1,[],[],[],[On the distinction between the first motion a...,"[[(On, None), (the, None), (distinction, None)..."
2,The testimony of the ages confirms that the mo...,[],[],1,[],[],[ Terms: 1. The first motion is that of the wh...,[The testimony of the ages confirms that the m...,"[[(The, None), (testimony, None), (of, None), ..."
3,It is just this from which astronomy arose amo...,[],[],1,[ ch 1 gr 1],[],[],[It is just this from which astronomy arose am...,"[[(It, None), (is, None), (just, None), (this,..."
4,Before the distinction between the first motio...,[],[(such],1,[],[],[ 2],[Before the distinction between the first moti...,"[[(Before, None), (the, None), (distinction, N..."


In [70]:
len(dfAstroNova)

1605

### Read the Sentences

In [71]:
df=dfAstroNova.sentences

In [72]:
sents=[]
for para in dfAstroNova.sentences:
    sents +=para

In [73]:
len(sents)

6699

## Annotation Based on Consistant format with Prodigy 

<fig size= 3>
Prodigy accept a specific format; a JSONL format (newline-delimited JSON). Entities and other highlighted spans of text can be defined in the "spans" property. A example could look like this dictionary:
</p>    
text': 'On 1595 October 30 at 8h 20m, they found Mars at 17° 48’ Taurus, with a diurnal motion of 22’ 54” ^15.',
  'spans': [{'start': 22, 'end': 29, 'label': 'TIME'}]}
</p>
which start and end's numbers refer to the position of entitiy  in the sentence. (here PARA)

    
<fig size>

In [74]:
def print_sents_entities(examples):
    for example in examples:
        if example["spans"]!=[]:
            sent,entity=example['text'], example['text'][example["spans"][0]["start"]:example["spans"][0]["end"]]
            print(sent,[entity])

In [75]:
def print_entities(examples):
    entities = []
    for example in examples:
        if example["spans"] != []:
            entities += [example['text'][example["spans"][0]["start"]:example["spans"][0]["end"]]]
            len(entities)
    print("There is {} entities with this detail\n{}".format(len(entities),entities))

## See Misaligned Tokens

<fig size= 3>
This step is very important, since if Pordigy faces with Misaligned tokens, we can see that before and try to adjust regex accordingly
<fig size>

In [76]:
def misaligned_token(examples):
    counter=0
    nlp = spacy.load("en_core_web_sm")  
    for example in examples:  
        doc = nlp(example["text"])
        for span in example["spans"]:
            char_span = doc.char_span(span["start"], span["end"])
            if char_span is None:  
                counter+=1
                print("{}- Misaligned tokens-->".format(counter), example["text"], span)

# Edge Cases

As it is clear from above , After using regular expression we faced with some specific format like these words:
- 12° 38’ Pisces12
- 0° 7’ Virgo,7
- 8° 37’ Gemini,3
- 16° 47’ Cancer,6
- 12° 38’ Pisces,18
- 16° 43' Taurus,2
- 29° Sagittarius,4
- 11° 52',2
- 1° 37',32
- 11° 52',2
- 16° 43' Taurus,2
- 16° 43' Taurus10
- 0° 3' Leo26
- 11° 4’ Sagittarius,37
- 3° 5' Pisces,41
- 8° 36’ 50” Gemini2
- 28° 1’ 0” Libra,27
- 26° 26’ 17“ Virgo,8
- 3° 58’ 20“ Scorpio,13
- 27° 14’ 42“ Sagittarius,16
- 17° 47’ 15“ Taurus,19
- 37° 19’ 50“,24
- 1° 7' 5"-
- 26° 13' 39",2
- 7° 6' 23",11
- 6° 52' 20",4
- 3° 45' 45",4
- 7° 33' 20",8

We consider these words as edge cases which means tokens are not consistant with the tokens assigned by the model’s tokenizer.  

## Solution

We can modify our annotation rules based on the above examples

# Refined Edge Cases

- 8° 36’ 50” Gemini2



### One Possible Solution 

In [77]:
label = "COOR"   
texts = sents  

regex_patterns = [
                  re.compile(r"8° 36’ 50” Gemini2|87° 16\' 30\"8|16° 43\' Taurus10|0° 3\' Leo26|11° 52\',2|12° 38’ Pisces12"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]\s?[A-Z][a-z.]+(,\d{1,2})?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\'|’|\"|”|“]\s?\s?[A-Z][a-z.]+(,\d{1,2})?"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]-?(,\d{1,2})?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\'|’|\"|”|“](,\d{2})?"
                              "|\d{1,3}\s?°\s?[A-Z][a-z.]+(,\d{1,2})?")           
]     


regex_patterns = [
                  re.compile(r"28° 2½’,4|4° 32⅙’22|4° 32⅙’22|4° 2½’—supposing|3° 59½’,so|22° 21’ 31“Aquarius|8° 36’ 50” Gemini2|87° 16\' 30\"8|16° 43\' Taurus10|0° 3\' Leo26|11° 52\',2|12° 38’ Pisces12"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\'|’|\"|”|“]\s?([A-Z][a-z.]+|southern|south|north)(,\d{1,2})?"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]\s?([A-Z][a-z.]+|southern|south|north)(,\d{1,2})?"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]-?(,\d{1,2})?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\'|’|\"|”|“](,\d{2})?"
                              "|\d{1,3}\s?[\u00BC-\u00BE\u2150-\u215E]?°\s?([A-Z][a-z.]+|southern|south|north)(,\d{1,2})?")
]
 
#(5° 20\' 2\"  Libra,|21° 44\' 56\"  Sagittarius,|(?<!\S)\d{1,2}\s?°\s\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”]\s[A-Z][a-z]+(?!\S?\S)|(?<!\S)\d{1,2}\s?°\s\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”].?(?!\S?\S)|(?<!\S)\d{1,2}\s?°\s?\d{1,2}\s?[\'|’]\s?[A-Z][a-z]+(?!\S?\S))[,:/.]?
examples = []
for text in texts:
    for expression in regex_patterns:
        spans = []
    for match in re.finditer(expression, text):
        start, end = match.span()
        span = {"start": start, "end": end, "label": label}
        spans.append(span)
    task = {"text": text, "spans": spans}
    examples.append(task)              

write_jsonl("NER_COOR_V04.jsonl", examples)



In [78]:
a_10=print_entities(examples)

There is 1096 entities with this detail
['3° 4\' 52"', '41° 55\' 8"', '16° 33\' 30"', '7° 28\' 56"', '37° 32\' 37"', '1\' 33"', '4° 20\' 8"', '85° 39\' 52"', '11° 25\' 48"', '11° 23\' 53"', '1’ 55”', '47° 59¼’', '5° 27’ 47”', '23° 32\' 16" Leo', '5° 27’ 47”', '5° 27’ 47”', '78° 51’ 54”', '4’ 24”', '4’ 24”', '47° 59’ 16”', '44° 59’ 10”', '44° 59’ 10”', '53° 26’ 17”', '44° 59’ 10”', '7° 30’ 10”', '3° 0’ 6”', '8½° Cancer', '16° Cancer', '1° 3’ 32” But', '2° 3’', '9° Leo.', '23° 45’ Leo.', '5° 2’', '28° 2½’,4', '23° 20’ Leo', '17° Taurus', '3° Cancer', '17° Cancer', '27° Sagittarius.', '26° 45’ 24” Gemini.', '26° 24’ 43” Sagittarius.', '10’ 20”', '26° 24’ 13” Sagittarius', '18° Taurus', '13° Pisces.', '18° Taurus', '17° Taurus', '4° 33’', '1° 50’', '8° 36’ 50” Gemini2', '8° 20’ Gemini', '1° 55’', '1° 52’', '6° 41’ 50” Gemini', '1’ 4”', '6° 46’ 10” Gemini.', '16° 47’ Cancer', '16° 46’ 16” Cancer', '21° 18’ 11” Leo', '24’ 15”', '7’ 41”', '21° 10’ 30” Leo', '25° 10’ 20” Virgo.', '1’ 48”', '25

In [79]:
misaligned_token(examples)

In [80]:
a_09=print_entities(examples)

There is 1096 entities with this detail
['3° 4\' 52"', '41° 55\' 8"', '16° 33\' 30"', '7° 28\' 56"', '37° 32\' 37"', '1\' 33"', '4° 20\' 8"', '85° 39\' 52"', '11° 25\' 48"', '11° 23\' 53"', '1’ 55”', '47° 59¼’', '5° 27’ 47”', '23° 32\' 16" Leo', '5° 27’ 47”', '5° 27’ 47”', '78° 51’ 54”', '4’ 24”', '4’ 24”', '47° 59’ 16”', '44° 59’ 10”', '44° 59’ 10”', '53° 26’ 17”', '44° 59’ 10”', '7° 30’ 10”', '3° 0’ 6”', '8½° Cancer', '16° Cancer', '1° 3’ 32” But', '2° 3’', '9° Leo.', '23° 45’ Leo.', '5° 2’', '28° 2½’,4', '23° 20’ Leo', '17° Taurus', '3° Cancer', '17° Cancer', '27° Sagittarius.', '26° 45’ 24” Gemini.', '26° 24’ 43” Sagittarius.', '10’ 20”', '26° 24’ 13” Sagittarius', '18° Taurus', '13° Pisces.', '18° Taurus', '17° Taurus', '4° 33’', '1° 50’', '8° 36’ 50” Gemini2', '8° 20’ Gemini', '1° 55’', '1° 52’', '6° 41’ 50” Gemini', '1’ 4”', '6° 46’ 10” Gemini.', '16° 47’ Cancer', '16° 46’ 16” Cancer', '21° 18’ 11” Leo', '24’ 15”', '7’ 41”', '21° 10’ 30” Leo', '25° 10’ 20” Virgo.', '1’ 48”', '25